# Data compilation from database scan

The trained models have been used for scaning the whole database looking for incentives.\
Each sentence has been firs classified into one of to categories, *Incentive* or *Not-Incentive*.\
Sentences which were classified as incentives were further classified into one of the six policy instruments.

Thus 
* Number of sentences per document
* Average length of sentences
* Length of smallest sentence
* Length of longest sentence
* Relative distance of the first incentive from the head of the document
* Distance to the closest previous incentive


In [1]:
import boto3
import csv
import json

In [2]:
# path = 'C:/Users/jordi/Documents/claus/'
path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [3]:
def sentence_stats(sentence_text, min_sentence_length, max_sentence_length, sum_sentence_length):
    sum_sentence_length = sum_sentence_length + len(sentence_text)
    if len(sentence_text) < min_sentence_length:
        min_sentence_length = len(sentence_text)
    if len(sentence_text) > max_sentence_length:
        max_sentence_length = len(sentence_text)
    return min_sentence_length, max_sentence_length, sum_sentence_length


# def sentences_processing()    

In [4]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/input/"
file_name = "ElSalvadorKeys.json"
file = path + file_name

with open(file, 'r') as fp:
    SV_keys = json.load(fp)

In [6]:
language = "spanish"
model = "simple_transformers/"#""
in_prefix = f"{language}_documents/HSSC/{model}sentences/"
out_prefix = f"{language}_documents/HSSC/{model}updated_sentences/"

i = 0
j = 0
documents_with_incentives = 0
results_list = [["file_name", "n_incentives", "Supplies", "Loan", "Fine", "Direct payment", "Technical assistance", "Tax benefit"]]
incentives_dict = {}

for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):#.:
    if ".json" in obj.key and "ing" not in obj.key:
        i += 1
        name = obj.key.split("/")[-1].split("_")[0]
        if i % 1000 == 0:
            print(i,  end='\r')
        if name in SV_keys:
            j += 1
            print(name)
            
#           output.clear()
#         if i < 100000:
    #         print(f"\n\nFile number: {i}\n\n")
#             print(i,  end='\r')
#             name = obj.key.split("/")[-1].split("_")[0]
            flag = False
            sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
            incentive_counter = 0
            sentences_counter = 0
            min_sentence_length = 100000
            max_sentence_length = 0
            sum_sentence_length = 0
            last_sentence_position = 0
            incentives_dict = {'Supplies': 0,
                               'Loan': 0,
                               'Fine': 0,
                               'Direct payment': 0,
                               'Technical assistance': 0,
                               'Tax benefit': 0}
            for sentence_id in sentences[name]["sentences"]:
                sentences_counter += 1
                sentence = sentences[name]["sentences"][sentence_id]["text"]
                min_sentence_length, max_sentence_length, sum_sentence_length = sentence_stats(sentence, min_sentence_length, max_sentence_length, sum_sentence_length)
                incentive = sentences[name]["sentences"][sentence_id]["label"]  
                if len(incentive) > 1 and incentive[0] == "Incentive":
                    incentive_counter += 1
                    sentence_number = sentence_id.split("_")[2]
                    policy = sentences[name]["sentences"][sentence_id]["label"][1]
                    if last_sentence_position == 0:
                        results_list.append([])
                        last_sentence_position = sentence_number
                        sentences[name]["sentences"][sentence_id]["closest_sentence"] = 0
                        flag = True
                    else:
                        sentences[name]["sentences"][sentence_id]["closest_sentence"] = int(sentence_number) - int(last_sentence_position)
                        last_sentence_position = sentence_number

                    relative_position = int(sentence_number) / sentences[name]["metadata"]["n_sentences"]
                    sentences[name]["sentences"][sentence_id]["relative_position"] = round(relative_position, 2)
                    incentives_dict[policy] += 1
    #                 print(incentive, "\n", sentence_id, " ** ", last_sentence_position)

            # Updating the dictionary
            sentences[name]["metadata"]["min_sent_length"] = min_sentence_length
            sentences[name]["metadata"]["max_sent_length"] = max_sentence_length
            sentences[name]["metadata"]["avg_sent_length"] = int(sum_sentence_length/sentences_counter)
            sentences[name]["metadata"]["number_of_incentives"] = incentive_counter
            for key, value in incentives_dict.items():
                sentences[name]["metadata"][key] = value
            s3.Object(s3BucketName, f"{out_prefix}{name}_sents.json").put(Body = (json.dumps(sentences, indent = 4)))
#             # Creating the results list
#             if flag:
#                 documents_with_incentives += 1
#                 results_list[documents_with_incentives].append(name)
#                 results_list[documents_with_incentives].append(incentive_counter)
#                 for key, value in incentives_dict.items():
#                     results_list[documents_with_incentives].append(value)

# with open(f"../output/incentives_simple_transformers_{language}.csv", "w") as f:
#     writer = csv.writer(f)
#     writer.writerows(results_list)

00a55afe4f55256567397a68df5d7f97e642480b
03030d8c8a358983da0caa6451ac3509b2097032
03a83f80282f03c903ffb2b01b29c5157c09ecc3
05f5ba472db01283102638322737972ad1ddca21
06dafc0a507e270d9b23b192489cb1012aed1a8f
07eec23ae8be1f90e93bab94fffab77d46871b9c
081affee12e9cd3823f6e692f4b245d7b426e040
092e4c7a8b112c89e5a7bbe5847ddaa3c25e861b
0bee7f897ed7d3d2a8298b1006d9167d6f6d6dcc
0c5f5ad93e47d014f1c1f9a3c7d2fe4a463d1099
0cc5362b18c38596d98f51a2bc3f5aeacfae0eba
0d8a4cee0fd8458ab529a19c37844de79cd4305f
0eab4fd43506361db1ada8eee32a944b12157677
0f03aa3518c8e40d930ab2e6381f7ad48d7f0328
0febf9d3977fee6ea71cdc8216dae059f2dc9dde
1123ffefcc396ddc15a20fc1f73edbabd1b0fea5
135664a42be852605f1aa6f90530aaa33def856b
15169f795b04dd22b4bb7176b1a33d0daae1af1a
1555e55d19bbe97096829f63bfe8ec95bf6257fe
161d1f4dd27cd2b2b44d109a0b0c337ad07d3fac
18d1b46d5a79f45b7e03b9b0833370024846fc5a
19b9432d1d9fa1b95f28c93c8710ff87538da2d5
1a77612f3b8b53c2b6762896f690af0ab02a5c44
1c909b8c7af0d738e4ad2380339732c720c6fc3f
1cd36a07cb429ffa

bcf7f64ed5d7aa94ce6b7b25fbd93015c4a465b2
be1bd90d183ad911a7d230d54b73dbd0ae5f0dde
becb2891e83971c77bbf032251e83351a533c510
beedd83da8916ac38026697d829f61b5e2c0282c
bf78089f6178944158517f27b9b7cd01c1979c75
bfd12f9c04d693956227d38e24abbfacf3e83ad4
c00e1d3a9433503e81be2a91e32ce82f1370dbd5
c06de9d12e2036352cae7577f5ec4029ba2c8ab5
c1f8639ffc2ff69e408593d3a76e6c7dc69fb058
c22afdc3b008e3c069ce794a9e6a0ec6cf4cad98
c3f28068e6c0e3a782b21d961073681e2fb5fc76
c56175ec33fd36f76e8f1c2d15dfcc64c5d76b39
c6e5f1dc89f6943598ac3c69026779a7fe8bece2
c85c73b5e8950f8d6de62c3eb07b9b443553edd9
c89d85ebb983816ea0288a4e62e5cba5820fd20d
c974cca2e41a45eec7caabfa8dbeb5911dd2dc41
ca4e78d73898cfdd1374d5aa6b7a3ec40d8f732f
cad84c955dbe0c17472b8211a667f3bea3479429
cc92822e8ab45789c755a7b6894fe00cb81a6c1a
cd11969f9528064373def79ef268f27c1c5da6cb
ce1b45c3e048e48484025f4cbb883f47659140f3
ce2bd5586b217e8ddc7eae8f68d9d7ef3404c880
ce5cd8fa7bffc00a57a3e07f756087d82da44dbf
cefe040ddbb63dec7b3301b5d86f894899c4ba23
cfe0ee34a32cd0ab

In [ ]:
j

In [ ]:
incentives_dict

In [ ]:
results_list

In [ ]:
sentences